In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
import math
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
import gcsfs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 13 12:02 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
bucket_name = '208906255' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.startswith("multi"):  # Reads only the data files and not the indices
        paths.append(full_path+b.name)

# Importing global stats

In [ ]:
# Importing df_dict fro bucket
file_name = 'w2df_dict.pkl'
# Create a GCSFileSystem object
fs = gcsfs.GCSFileSystem()
# Open the file from the bucket
bucket_name = '208906255'
with fs.open(f'gs://208906255/global_stats/w2df_dict.pkl', 'rb') as f:
    # Load the pickle file
    w2df_dict = pickle.load(f)

In [ ]:
# Importing dl_dict fro bucket
file_name = 'd2dl_dict.pkl'
# Create a GCSFileSystem object
fs = gcsfs.GCSFileSystem()
# Open the file from the bucket
bucket_name = '208906255'
with fs.open(f'gs://208906255/global_stats/d2dl_dict.pkl', 'rb') as f:
    # Load the pickle file
    d2dl_dict = pickle.load(f)

In [ ]:
# Importing dl_dict fro bucket
file_name = 'd2dl_dict.pkl'
# Create a GCSFileSystem object
fs = gcsfs.GCSFileSystem()
# Open the file from the bucket
bucket_name = '208906255'
with fs.open(f'gs://208906255/global_stats/d2dl_dict.pkl', 'rb') as f:
    # Load the pickle file
    d2dl_dict = pickle.load(f)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)

In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("id", "title").rdd
doc_anchor_pairs = parquetFile.select("id", "anchor_text").rdd

In [ ]:
# Working on the id_title dictionary
doc_title_pairs = parquetFile.select("id", "title").rdd
doc_title_pairs = doc_title_pairs.map(lambda x: (x.id, x.title))

# Title Index

In [ ]:
#creating title index
doc_title_pairs = doc_title_pairs.map(lambda x: (x.id, x.title))

In [ ]:
title_counts = doc_title_pairs.flatMap(lambda x: word_count(x[1], x[0]))

In [ ]:
title_postings = title_counts.groupByKey().mapValues(reduce_word_counts)

In [ ]:
title_w2df = calculate_df(title_postings)
title_w2df_dict = title_w2df.collectAsMap()

In [ ]:
#creating DL dictionary
title_DL = title_counts.map(lambda x: x[1])
title_len = title_DL.reduceByKey(lambda a,b: a+b)
title_d2dl_dict = title_len.collectAsMap()

In [ ]:
#save the title_d2dl_dict
with open('title_d2dl_dict.pkl', 'wb') as f:
    pickle.dump(title_d2dl_dict, f)
title_d2dl_dict_src = "title_d2dl_dict.pkl"
title_d2dl_dict_dst = f'gs://{bucket_name}/global_stats/{title_d2dl_dict_src}'  
!gsutil cp $title_d2dl_dict_src $title_d2dl_dict_dst

In [ ]:
# partition posting lists and write out
_ = partition_postings_and_write(title_postings).collect()

In [ ]:
def title_calculate_idf(postings):
    return postings.map(lambda x: (x[0],math.log(N/title_w2df_dict.get(x[0],0),10)))  # Creates tuples of (word, idf) for each word

def title_norm_doc(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    filtered_words = [word for word in tokens if word not in all_stopwords]  # Filters all the stop words
    terms_dict = Counter(filtered_words)  # Dictionary for term and its tf
    term_tf_list = terms_dict.items()  # List of tuples of (term, tf)
    words_id_tf_list = [((id, term_tf[0]), term_tf[1]) for term_tf in term_tf_list if title_w2df_dict.get(term_tf[0],-10) >=0 ]  #((id,term),tf)
    tf_idf_square = list(map(lambda x: math.pow(((x[1]/title_d2dl_dict[id])*math.log(N/title_w2df_dict[x[0][1]],10)),2),words_id_tf_list)) #sum((tf-idf)^2)
    s = 0
    for w in tf_idf_square:
        s += w
    return (id,math.sqrt(s))

In [ ]:
w2idf_title = title_calculate_idf(title_postings)
title_w2idf_dict = w2idf_title.collectAsMap()

In [ ]:
# save the title_d2dl_dict
with open('title_w2idf_dict.pkl', 'wb') as f:
    pickle.dump(title_w2idf_dict, f)
title_w2idf_dict_src = "title_w2idf_dict.pkl"
title_w2idf_dict_dst = f'gs://{bucket_name}/global_stats/{title_w2idf_dict_src}'  
!gsutil cp $title_w2idf_dict_src $title_w2idf_dict_dst

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = title_w2df_dict

inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/{index_src}'

!gsutil cp $index_src $index_dst

In [ ]:
title_norm2doc =  doc_title_pairs.map(lambda x: title_norm_doc(x[1], x[0]))
title_norm2doc_dict = title_norm2doc.collectAsMap()

In [ ]:
# save the title_d2dl_dict
with open('title_norm2doc_dict.pkl', 'wb') as f:
    pickle.dump(title_norm2doc_dict, f)
title_norm2doc_dict_src = "title_norm2doc_dict.pkl"
title_norm2doc_dict_dst = f'gs://{bucket_name}/global_stats/{title_norm2doc_dict_src}'  
!gsutil cp $title_norm2doc_dict_src $title_norm2doc_dict_dst

# Anchor index

In [ ]:
#creating anchor index
anchor_id_text = doc_anchor_pairs.flatMap(lambda x: x.anchor_text)
id_text = anchor_id_text.map(lambda x: (x.text, x.id))
anchor_counts = id_text.flatMap(lambda x: word_count(x[0], x[1]))
# Example: [('political', (23040, 1)), ('philosophy', (23040, 1)), ('movement', (99232, 1))]
anchor_postings = anchor_counts.groupByKey().mapValues(reduce_word_counts)

In [ ]:
anchor_w2df = calculate_df(anchor_postings)
anchor_w2df_dict = anchor_w2df.collectAsMap()

In [ ]:
_ = partition_postings_and_write(anchor_postings).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = anchor_w2df_dict

inverted.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"

index_dst = f'gs://{bucket_name}/anchor_index/{index_src}'

!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 95.1 MiB/ 95.1 MiB]                                                
Operation completed over 1 objects/95.1 MiB.                                     


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
N = parquetFile.count()

import the inverted index module

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token,offset=1):
    return int(_hash(token),16) % NUM_BUCKETS + (NUM_BUCKETS*offset)

def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    filtered_words = [word for word in tokens if word not in all_stopwords]  # Filters all the stop words
    terms_dict = Counter(filtered_words)  # Dictionary for term and its tf
    term_tf_list = terms_dict.items()  # List of tuples of (term, tf)
    words_id_tf_list = [(term_tf[0], (id, term_tf[1])) for term_tf in term_tf_list]

    return words_id_tf_list

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl)

def calculate_df(postings):
    return postings.map(lambda x: (x[0], len(x[1])))  # Creates tuples of (word, df) for each word

def partition_postings_and_write(postings):
    postings_buckets = postings.map(lambda x: (token2bucket_id(x[0]), x))  # (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...]) 
    postings_buckets = postings_buckets.groupByKey()  # Group tuples with identical keys
    posting_lists_by_buckets = postings_buckets.map(lambda x: (x[0], list(x[1])))
    word_location = posting_lists_by_buckets.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))

    return word_location

def calculate_idf(postings):
    return postings.map(lambda x: (x[0],math.log(N/w2df_dict.get(x[0],0),10)))  # Creates tuples of (word, idf) for each word


def norm_doc(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    filtered_words = [word for word in tokens if word not in all_stopwords]  # Filters all the stop words
    terms_dict = Counter(filtered_words)  # Dictionary for term and its tf
    term_tf_list = terms_dict.items()  # List of tuples of (term, tf)
    words_id_tf_list = [((id, term_tf[0]), term_tf[1]) for term_tf in term_tf_list if w2df_dict.get(term_tf[0],-10) >=0 ]  #((id,term),tf)
    tf_idf_square = list(map(lambda x: math.pow(((x[1]/d2dl_dict[id])*math.log(N/w2df_dict[x[0][1]],10)),2),words_id_tf_list)) #sum((tf-idf)^2)
    s = 0
    for w in tf_idf_square:
        s += w
    return (id,math.sqrt(s))

In [ ]:
#continue creating the global dictionary of id_title
title_counts = doc_title_pairs.flatMap(lambda x: word_count(x[1], x[0]))

In [ ]:
#continue creating the global dictionary of id_title
id2title_dict = doc_title_pairs.collectAsMap()

In [ ]:
# #save the id2title_dicts
# with open('id2title_dict.pkl', 'wb') as f:
#     pickle.dump(id2title_dict, f)
# id_title_src = "id2title_dict.pkl"
# id_title_dst = f'gs://{bucket_name}/global_stats/{id_title_src}'  
# !gsutil cp $id_title_src $id_title_dst

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
# _ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
#creating DL dictionary
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
doc_DL = word_counts.map(lambda x: x[1])
doc_len = doc_DL.reduceByKey(lambda a,b: a+b)
d2dl_dict = doc_len.collectAsMap()

In [ ]:
# save the d2dl_dict
# with open('d2dl_dict.pkl', 'wb') as f:
#     pickle.dump(d2dl_dict, f)
# d2dl_dict_src = "d2dl_dict.pkl"
# d2dl_dict_dst = f'gs://{bucket_name}/global_stats/{d2dl_dict_src}'  
# !gsutil cp $d2dl_dict_src $d2dl_dict_dst

In [ ]:
# save the wtdf_dict
# with open('w2df_dict.pkl', 'wb') as f:
#     pickle.dump(w2df_dict, f)
# w2df_dict_src = "w2df_dict.pkl"
# w2df_dict_dst = f'gs://{bucket_name}/global_stats/{w2df_dict_src}'  
# !gsutil cp $w2df_dict_src $w2df_dict_dst

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_postings'): ##name of prefix is n
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
# inverted.df = w2df_dict
# write the global stats out
# inverted.write_index('.', 'index')
# inverted.write_index('.', 'body_index')  # we added this line
# inverted.write_index('.', 'title_index')  # we added this line
inverted.write_index('.', 'anchor_index')  # we added this line

# upload to gs
# index_src = "title_index.pkl"
# index_src = "body_index.pkl"  # we added this line
# index_src = "title_index.pkl"  # we added this line
index_src = "anchor_index.pkl"  # we added this line
index_dst = f'gs://{bucket_name}/anchor_postings/{index_src}'  # we added this line. need to create new bucket and put its name instead of postings_gcp


!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
##########################  ###########################

DOC-Norm dictionary for cosine similarity

In [ ]:
 doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
norm2doc =  doc_text_pairs.map(lambda x: norm_doc(x[0], x[1]))
norm2doc_dict = norm2doc.collectAsMap()

In [ ]:
# save the norm_dict
with open('norm2doc_dict.pkl', 'wb') as f:
ןגכnorm2doc_dict_src = "norm2doc_dict.pkl"
norm2doc_dict_dst = f'gs://{bucket_name}/global_stats/{norm2doc_dict_src}'  
!gsutil cp $norm2doc_dict_src $norm2doc_dict_dst

In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2idf = calculate_idf(postings_filtered)
w2idf_dict = w2idf.collectAsMap()

In [ ]:
# save the w2idf_dict
with open('w2idf_dict.pkl', 'wb') as f:
    pickle.dump(w2idf_dict, f)
w2idf_dict_src = "w2idf_dict.pkl"
w2idf_dict_dst = f'gs://{bucket_name}/global_stats/{w2idf_dict_src}'  
!gsutil cp $w2idf_dict_src $w2idf_dict_dst

# Importing Indices

In [ ]:
# Importing Body Index from Bucket 
file_name = 'index.pkl'
# Create a GCSFileSystem object
fs = gcsfs.GCSFileSystem()
# Open the file from the bucket
bucket_name = '208906255'
with fs.open(f'gs://208906255/postings_gcp/index.pkl', 'rb') as f:
    # Load the pickle file
    body_index = pickle.load(f)

In [ ]:
# Importing Anchor Index from Bucket 
file_name = 'anchor_index.pkl'
# Create a GCSFileSystem object
fs = gcsfs.GCSFileSystem()
# Open the file from the bucket
bucket_name = '208906255'
with fs.open(f'gs://208906255/anchor_index/anchor_index.pkl', 'rb') as f:
    # Load the pickle file
    anchor_index = pickle.load(f)

In [ ]:
# Importing Title Index from Bucket 
file_name = 'title_index.pkl'
# Create a GCSFileSystem object
fs = gcsfs.GCSFileSystem()
# Open the file from the bucket
bucket_name = '208906255'
with fs.open(f'gs://208906255/title_index/title_index.pkl', 'rb') as f:
    # Load the pickle file
    title_index = pickle.load(f)

Page Rank

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
    edges = pages.flatMap(lambda page: [(page[0], anchor_text[0]) for anchor_text in page[1]])  # Tuples of (page_id, anchor page_id)
    edges = edges.distinct()  # Removes all the duplicates
    vertices = edges.flatMap(lambda id: id)
    vertices = vertices.distinct()  # Removes all the duplicates
    vertices = vertices.map(lambda id: [id])

    return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0 
print(f'I used {cost} USD credit during the course of this assignment')

In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag